## Validation with Field in Pydantic


#### 1. Basic Field Usage


In [22]:
from pydantic import BaseModel, Field


# Define a model using Field for metadata and defaults
class Item(BaseModel):
    # Field 1: Required, but with a clear title and description
    name: str = Field(
        ...,  # The Ellipsis (...) means this field is required (no default)
        title="Item Name",
        description="The descriptive name of the item.",
    )

    # Field 2: Optional, with a default value and descriptive metadata
    price: float = Field(
        0.0,
        title="Item Price",
        description="The price of the item in USD.",
        ge=0,  # ge (Greater than or Equal to) is a validation constraint
    )

In [23]:
# Instantiation with a default value
item1 = Item(name="Coffee Mug")

print(f"Item 1 - name: {item1.name}")
print(f"Item 1 - price: {item1.price}")

Item 1 - name: Coffee Mug
Item 1 - price: 0.0


In [24]:
# Invalid
item1 = Item(name="Coffee Mug", price=-1)

print(f"Item 1 - name: {item1.name}")
print(f"Item 1 - price: {item1.price}")

ValidationError: 1 validation error for Item
price
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than_equal

---

#### 2. String Validation Constraints


In [25]:
from pydantic import BaseModel, Field


class Post(BaseModel):
    # Must be between 5 and 50 characters long
    title: str = Field(
        ...,
        min_length=6,
        max_length=50,
    )

    # Must be exactly 3 characters long (e.g., a currency code)
    currency: str = Field(
        ...,
        min_length=3,
        max_length=3,
    )

In [26]:
# Valid post
post1 = Post(title="My First Pydantic Post", currency="USD")
print(f"Valid Post Title: {post1.title}")

Valid Post Title: My First Pydantic Post


In [27]:
# Invalid title (too short)
Post(title="Short", currency="EUR")

ValidationError: 1 validation error for Post
title
  String should have at least 6 characters [type=string_too_short, input_value='Short', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/string_too_short

In [28]:
# Invalid currency length
Post(title="A Normal Title", currency="C")

ValidationError: 1 validation error for Post
currency
  String should have at least 3 characters [type=string_too_short, input_value='C', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/string_too_short

---

#### 3. Numeric Validation Constraints


In [29]:
from pydantic import BaseModel, Field


class Product(BaseModel):
    # Price must be greater than 0
    price: float = Field(
        ...,
        gt=0,  # Greater Than 0
    )

    # Quantity must be an integer between 1 and 100, inclusive
    quantity: int = Field(
        ...,
        ge=1,  # Greater than or Equal to 1
        le=100,  # Less than or Equal to 100
    )

In [30]:
# Valid product
prod1 = Product(price=19.99, quantity=5)
print(f"Valid Price: {prod1.price}, Quantity: {prod1.quantity}")

Valid Price: 19.99, Quantity: 5


In [31]:
# Invalid price (not greater than 0)
Product(price=0, quantity=10)

ValidationError: 1 validation error for Product
price
  Input should be greater than 0 [type=greater_than, input_value=0, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than

In [32]:
# Invalid quantity (too high)
Product(price=10.0, quantity=101)

ValidationError: 1 validation error for Product
quantity
  Input should be less than or equal to 100 [type=less_than_equal, input_value=101, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/less_than_equal

---

#### 4. Aliases and Regular Expressions (Regex)


In [33]:
from pydantic import BaseModel, Field


class UserProfile(BaseModel):
    # Alias: Input data must use 'user_email', but the model uses 'email'
    email: str = Field(
        ...,
        alias="user_email",
        description="The primary email address of the user.",
    )

    # Regex: Must start with a letter and contain only alphanumeric/underscore (like a username)
    username: str = Field(
        ...,
        pattern=r"^[A-Za-z][A-Za-z0-9_]{3,15}$",
    )

In [34]:
# Valid data (using the alias 'user_email')
valid_data = {
    "user_email": "john.doe@example.com",
    "username": "JDoe_23",
}
user1 = UserProfile(**valid_data)
# Access the field using the model attribute name 'email'
print(f"Valid User Email: {user1.email}, Username: {user1.username}")

Valid User Email: john.doe@example.com, Username: JDoe_23


In [35]:
# Invalid data (violating the regex rule - starting with a number)
invalid_data = {
    "user_email": "test@test.com",
    "username": "123_invalid",
}  # Starts with a number

UserProfile(**invalid_data)

ValidationError: 1 validation error for UserProfile
username
  String should match pattern '^[A-Za-z][A-Za-z0-9_]{3,15}$' [type=string_pattern_mismatch, input_value='123_invalid', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/string_pattern_mismatch